In [1]:
sc

In [2]:
!rm ./metastore_db/*.lck
examples_folder = "/home/ubuntu/spark/examples/src/main/resources/"

In [3]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Parquet Files

In [4]:
df = sqlc.read.load(examples_folder + "users.parquet")
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



In [5]:
!rm -rf namesAndFavColors.parquet/
df.select("name", "favorite_color").write.save("namesAndFavColors.parquet")

In [6]:
!ls -l namesAndFavColors.parquet/

total 4
-rw-r--r-- 1 ubuntu ubuntu 572 Jan 24 12:28 part-00000-48237f36-a161-40f0-9a6e-be1e0de7825a.snappy.parquet
-rw-r--r-- 1 ubuntu ubuntu   0 Jan 24 12:28 _SUCCESS


In [7]:
sqlc.read.format("org.apache.spark.sql.parquet").load("namesAndFavColors.parquet")

DataFrame[name: string, favorite_color: string]

In [8]:
sqlc.read.format("parquet").load("namesAndFavColors.parquet")

DataFrame[name: string, favorite_color: string]

In [9]:
sqlc.read.parquet("namesAndFavColors.parquet")

DataFrame[name: string, favorite_color: string]

## JSON Files

In [10]:
df_json = sqlc.read.format("json").load(examples_folder + "people.json")
df_json.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [11]:
df_json.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [12]:
json_strings = df_json.toJSON()
json_strings.collect()

[u'{"name":"Michael"}',
 u'{"age":30,"name":"Andy"}',
 u'{"age":19,"name":"Justin"}']

In [13]:
!rm -rf namesAndAges.parquet/
df_json.select("name", "age").write.format("parquet").save("namesAndAges.parquet")

In [14]:
!ls -l namesAndAges.parquet

total 4
-rw-r--r-- 1 ubuntu ubuntu 585 Jan 24 12:28 part-00000-e91075df-0f69-4e59-a132-fc28ad3422be.snappy.parquet
-rw-r--r-- 1 ubuntu ubuntu   0 Jan 24 12:28 _SUCCESS


In [15]:
!rm -rf namesAndAgesCoalesced.parquet/
df_json.select("name", "age").coalesce(1).write.format("parquet").save("namesAndAgesCoalesced.parquet")

In [16]:
!ls -l namesAndAgesCoalesced.parquet

total 4
-rw-r--r-- 1 ubuntu ubuntu 585 Jan 24 12:28 part-00000-9815aeb8-35d7-4eaa-bfd5-4f3f70da0570.snappy.parquet
-rw-r--r-- 1 ubuntu ubuntu   0 Jan 24 12:28 _SUCCESS


In [17]:
df_json.write.mode("error").parquet("namesAndAges.parquet")
# This IS supposed to throw an exception, as it is trying to write an already existent file!

AnalysisException: u'path file:/home/ubuntu/AUDI-SPARK-TRAINING/namesAndAges.parquet already exists.;'

In [18]:
df_json.write.mode("overwrite").parquet("namesAndAges.parquet")

## SQL on Files

In [19]:
df = sqlc.sql("SELECT * FROM json.`/home/ubuntu/spark/examples/src/main/resources/people.json` WHERE age = 19")
df.show()

+---+------+
|age|  name|
+---+------+
| 19|Justin|
+---+------+



## MySQL

#### We need to run these commands in MYSQL to create the table:

```SQL
CREATE TABLE users (user_id int PRIMARY KEY, fname text, lname text);
INSERT INTO users (user_id,  fname, lname) VALUES (1, 'john', 'smith');
INSERT INTO users (user_id,  fname, lname) VALUES (2, 'john', 'doe');
INSERT INTO users (user_id,  fname, lname) VALUES (3, 'john', 'smith');
```

In [20]:
# Start the SQL server:

!/etc/init.d/mysql start

Starting mysql (via systemctl): mysql.service.


In [21]:
%%bash

# Run the above commands as root
mysql -u root --password=root << EOF
CREATE DATABASE IF NOT EXISTS db;
USE db;
DROP TABLE IF EXISTS users;
CREATE TABLE users (user_id int PRIMARY KEY, fname text, lname text);
INSERT INTO users (user_id,  fname, lname) VALUES (1, 'john', 'smith');
INSERT INTO users (user_id,  fname, lname) VALUES (2, 'john', 'doe');
INSERT INTO users (user_id,  fname, lname) VALUES (3, 'john', 'smith');
EOF

mysql: [Warning] Using a password on the command line interface can be insecure.


In [22]:
df_mysql = sqlc.read.format("jdbc") \
                .option("url", "jdbc:mysql://localhost:3306/db") \
                .option("driver", "com.mysql.jdbc.Driver") \
                .option("dbtable", "users") \
                .option("user", "root") \
                .option("password", "root").load()

In [23]:
df_mysql.show()

+-------+-----+-----+
|user_id|fname|lname|
+-------+-----+-----+
|      1| john|smith|
|      2| john|  doe|
|      3| john|smith|
+-------+-----+-----+



In [24]:
df_mysql.printSchema()

root
 |-- user_id: integer (nullable = false)
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)



In [25]:
df_mysql.registerTempTable("users")

In [26]:
sqlc.sql("select * from users").collect()

[Row(user_id=1, fname=u'john', lname=u'smith'),
 Row(user_id=2, fname=u'john', lname=u'doe'),
 Row(user_id=3, fname=u'john', lname=u'smith')]

In [27]:
sc

In [28]:
!rm ./metastore_db/*.lck

In [29]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## CSV

In [30]:
!wget https://github.com/databricks/spark-csv/raw/master/src/test/resources/cars.csv

--2017-01-24 12:28:50--  https://github.com/databricks/spark-csv/raw/master/src/test/resources/cars.csv
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/databricks/spark-csv/master/src/test/resources/cars.csv [following]
--2017-01-24 12:28:51--  https://raw.githubusercontent.com/databricks/spark-csv/master/src/test/resources/cars.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134 [text/plain]
Saving to: ‘cars.csv.2’

cars.csv.2          100%[===================>]     134  --.-KB/s    in 0s      

2017-01-24 12:28:51 (17,1 MB/s) - ‘cars.csv.2’ saved [134/134]



In [31]:
df_cars = sqlc.read.format("com.databricks.spark.csv") \
                .option("header", "true") \
                .option("inferSchema", "true") \
                .load("cars.csv")

In [32]:
df_cars.show()

+----+-----+-----+--------------------+-----+
|year| make|model|             comment|blank|
+----+-----+-----+--------------------+-----+
|2012|Tesla|    S|          No comment| null|
|1997| Ford| E350|Go get one now th...| null|
|2015|Chevy| Volt|                null| null|
+----+-----+-----+--------------------+-----+



In [33]:
df_cars.printSchema()

root
 |-- year: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- blank: string (nullable = true)



In [34]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

customSchema = StructType([StructField("year", StringType(), True),
                           StructField("make", StringType(), True),
                           StructField("model", StringType(), True), 
                           StructField("comment", StringType(), True),
                           StructField("blank", StringType(), True)])

In [35]:
df_cars2 = sqlc.read.load(path="cars.csv", 
                          format="com.databricks.spark.csv", 
                          schema=customSchema,
                          header=True)

In [36]:
df_cars2.printSchema()

root
 |-- year: string (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- blank: string (nullable = true)



In [37]:
!rm -rf newcars.csv

selectedData = df_cars.select("year", "model","comment")
selectedData.coalesce(1).write.format("com.databricks.spark.csv") \
                        .option("header", "true") \
                        .option("nullValue","NA") \
                        .save("newcars.csv") \

In [38]:
!ls -l newcars.csv

total 4
-rw-r--r-- 1 ubuntu ubuntu 95 Jan 24 12:29 part-00000-e4a4e5b0-448d-41cb-bc7b-cca88c6ed50c.csv
-rw-r--r-- 1 ubuntu ubuntu  0 Jan 24 12:29 _SUCCESS


In [39]:
!rm -rf newcars.csv.gz
selectedData.write.format("com.databricks.spark.csv") \
                    .option("header", "true") \
                    .option("codec", "gzip") \
                    .save("newcars.csv.gz")

In [40]:
!ls -l newcars.csv.gz

total 4
-rw-r--r-- 1 ubuntu ubuntu 104 Jan 24 12:29 part-00000-c4daed4d-6af4-4eeb-b42a-64edb357ae29.csv.gz
-rw-r--r-- 1 ubuntu ubuntu   0 Jan 24 12:29 _SUCCESS


## XML

In [41]:
!wget https://github.com/databricks/spark-xml/raw/master/src/test/resources/books.xml

--2017-01-24 12:29:07--  https://github.com/databricks/spark-xml/raw/master/src/test/resources/books.xml
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/databricks/spark-xml/master/src/test/resources/books.xml [following]
--2017-01-24 12:29:08--  https://raw.githubusercontent.com/databricks/spark-xml/master/src/test/resources/books.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5542 (5,4K) [text/plain]
Saving to: ‘books.xml.2’

books.xml.2         100%[===================>]   5,41K  --.-KB/s    in 0s      

2017-01-24 12:29:08 (109 MB/s) - ‘books.xml.2’ saved [5542/5542]



In [42]:
!cat books.xml

<?xml version="1.0"?>
<catalog>
   <book id="bk101">
      <author>Gambardella, Matthew</author>
      <title>XML Developer's Guide</title>
      <genre>Computer</genre>
      <price>44.95</price>
      <publish_date>2000-10-01</publish_date>
      <description>


         An in-depth look at creating applications
         with XML.This manual describes Oracle XML DB, and how you can use it to store, generate, manipulate, manage,
         and query XML data in the database.


         After introducing you to the heart of Oracle XML DB, namely the XMLType framework and Oracle XML DB repository,
         the manual provides a brief introduction to design criteria to consider when planning your Oracle XML DB
         application. It provides examples of how and where you can use Oracle XML DB.


         The manual then describes ways you can store and retrieve XML data using Oracle XML DB, APIs for manipulating
         XMLType data, and ways you can view, generate, transform, and searc

In [43]:
df_books = sqlc.read.format("com.databricks.spark.xml") \
                    .option("rowTag", "book") \
                    .load("books.xml")

In [44]:
df_books.printSchema()

root
 |-- _id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- description: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- publish_date: string (nullable = true)
 |-- title: string (nullable = true)



In [45]:
df_books.show()

+-----+--------------------+--------------------+---------------+-----+------------+--------------------+
|  _id|              author|         description|          genre|price|publish_date|               title|
+-----+--------------------+--------------------+---------------+-----+------------+--------------------+
|bk101|Gambardella, Matthew|


         An in...|       Computer|44.95|  2000-10-01|XML Developer's G...|
|bk102|          Ralls, Kim|A former architec...|        Fantasy| 5.95|  2000-12-16|       Midnight Rain|
|bk103|         Corets, Eva|After the collaps...|        Fantasy| 5.95|  2000-11-17|     Maeve Ascendant|
|bk104|         Corets, Eva|In post-apocalyps...|        Fantasy| 5.95|  2001-03-10|     Oberon's Legacy|
|bk105|         Corets, Eva|The two daughters...|        Fantasy| 5.95|  2001-09-10|  The Sundered Grail|
|bk106|    Randall, Cynthia|When Carla meets ...|        Romance| 4.95|  2000-09-02|         Lover Birds|
|bk107|      Thurman, Paula|A deep sea diver .

In [46]:
!rm -rf newbooks.xml

selectedData = df_books.select("author", "_id")
selectedData.write.format("com.databricks.spark.xml") \
                .option("rootTag", "books") \
                .option("rowTag", "book") \
                .save("newbooks.xml")

In [47]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

customSchema = StructType([StructField("_id", StringType(), nullable = True), 
                           StructField("author", StringType(), nullable = True),
                           StructField("description", StringType(), nullable = True),
                           StructField("genre", StringType(),nullable = True), 
                           StructField("price", DoubleType(), nullable = True),
                           StructField("publish_date", StringType(), nullable = True),
                           StructField("title", StringType(), nullable = True)])

In [48]:
df_books = sqlc.read.format("com.databricks.spark.xml") \
                    .option("rowTag", "book") \
                    .schema(customSchema) \
                    .load("books.xml")
            
selectedData = df_books.select("author", "_id")
selectedData.write.format("com.databricks.spark.xml") \
                .option("rootTag", "books") \
                .option("rowTag", "book") \
                .mode("overwrite") \
                .save("newbooks.xml")

In [49]:
sc

In [50]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [51]:
from collections import namedtuple
Sales = namedtuple("Sales",["id","account","year","commission","sales_reps"])

sales = sc.parallelize([Sales(1, "Acme", "2013", 1000, ["Jim", "Tom"]),
         Sales(2, "Lumos", "2013", 1100, ["Fred", "Ann"]),
         Sales(3, "Acme", "2014", 2800, ["Jim"]),
         Sales(4, "Lumos", "2014", 1200, ["Ann"]),
         Sales(5, "Acme", "2014", 4200, ["Fred", "Sally"])]).toDF()

sales.show()

+---+-------+----+----------+-------------+
| id|account|year|commission|   sales_reps|
+---+-------+----+----------+-------------+
|  1|   Acme|2013|      1000|   [Jim, Tom]|
|  2|  Lumos|2013|      1100|  [Fred, Ann]|
|  3|   Acme|2014|      2800|        [Jim]|
|  4|  Lumos|2014|      1200|        [Ann]|
|  5|   Acme|2014|      4200|[Fred, Sally]|
+---+-------+----+----------+-------------+



In [52]:
from pyspark.sql.functions import explode

sales.select("id","account","year","commission",explode("sales_reps").alias("sales_rep")).show()

+---+-------+----+----------+---------+
| id|account|year|commission|sales_rep|
+---+-------+----+----------+---------+
|  1|   Acme|2013|      1000|      Jim|
|  1|   Acme|2013|      1000|      Tom|
|  2|  Lumos|2013|      1100|     Fred|
|  2|  Lumos|2013|      1100|      Ann|
|  3|   Acme|2014|      2800|      Jim|
|  4|  Lumos|2014|      1200|      Ann|
|  5|   Acme|2014|      4200|     Fred|
|  5|   Acme|2014|      4200|    Sally|
+---+-------+----+----------+---------+



In [53]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType

column_len = UserDefinedFunction(lambda x: len(x), IntegerType())

exploded = sales.select("id","account","year","commission",
             column_len("sales_reps").alias("num_reps"),
             explode("sales_reps").alias("sales_rep"))

exploded.show()

+---+-------+----+----------+--------+---------+
| id|account|year|commission|num_reps|sales_rep|
+---+-------+----+----------+--------+---------+
|  1|   Acme|2013|      1000|       2|      Jim|
|  1|   Acme|2013|      1000|       2|      Tom|
|  2|  Lumos|2013|      1100|       2|     Fred|
|  2|  Lumos|2013|      1100|       2|      Ann|
|  3|   Acme|2014|      2800|       1|      Jim|
|  4|  Lumos|2014|      1200|       1|      Ann|
|  5|   Acme|2014|      4200|       2|     Fred|
|  5|   Acme|2014|      4200|       2|    Sally|
+---+-------+----+----------+--------+---------+



In [54]:
exploded = exploded.withColumn("share", exploded.commission / exploded.num_reps).drop("num_reps")
exploded.show()

+---+-------+----+----------+---------+------+
| id|account|year|commission|sales_rep| share|
+---+-------+----+----------+---------+------+
|  1|   Acme|2013|      1000|      Jim| 500.0|
|  1|   Acme|2013|      1000|      Tom| 500.0|
|  2|  Lumos|2013|      1100|     Fred| 550.0|
|  2|  Lumos|2013|      1100|      Ann| 550.0|
|  3|   Acme|2014|      2800|      Jim|2800.0|
|  4|  Lumos|2014|      1200|      Ann|1200.0|
|  5|   Acme|2014|      4200|     Fred|2100.0|
|  5|   Acme|2014|      4200|    Sally|2100.0|
+---+-------+----+----------+---------+------+



In [55]:
exploded.groupBy("sales_rep").pivot("year").sum("share").orderBy("sales_rep").show()

+---------+-----+------+
|sales_rep| 2013|  2014|
+---------+-----+------+
|      Ann|550.0|1200.0|
|     Fred|550.0|2100.0|
|      Jim|500.0|2800.0|
|    Sally| null|2100.0|
|      Tom|500.0|  null|
+---------+-----+------+



In [56]:
exploded.groupBy("account", "sales_rep").pivot("year").sum("share").orderBy("account", "sales_rep").show()

+-------+---------+-----+------+
|account|sales_rep| 2013|  2014|
+-------+---------+-----+------+
|   Acme|     Fred| null|2100.0|
|   Acme|      Jim|500.0|2800.0|
|   Acme|    Sally| null|2100.0|
|   Acme|      Tom|500.0|  null|
|  Lumos|      Ann|550.0|1200.0|
|  Lumos|     Fred|550.0|  null|
+-------+---------+-----+------+

